Forming en <-> zhtw alignment instruction-response dataset in Openai format.



In [5]:
SYS_PROMPT_EN2TW = [
    "請用繁體中文回答以下問題",
    "Please answer the following question in Traditional Chinese",
    "用繁體中文解釋下述指令",
    "Explain the following instructions in Traditional Chinese",
    "以繁體中文提供以下訊息",
    "Provide the following information in Traditional Chinese",
    "用繁體中文進行回應",
    "Respond in Traditional Chinese",
    "用繁體中文回答，問題如下",
    "Answer in Traditional Chinese, the question is as follows",
    "以繁體中文完成此任務",
    "Complete this task in Traditional Chinese"
]

In [6]:
from datasets import load_dataset, load_from_disk
import os
import random
import json
random.seed(11944004)
instruction_base_dir = "./instruction_tuning"
os.makedirs(instruction_base_dir, exist_ok=True)

In [7]:
# Alpaca GPT3.5-zhtw
alpaca_tw2tw = load_dataset("json", data_files="./alpaca_zhtw/alpaca-tw.json")
alpaca_en2tw = load_dataset("json", data_files="./alpaca_zhtw/alpaca-tw_en_instruction.json")

Generating train split: 52002 examples [00:00, 83660.69 examples/s] 


In [8]:
SYS_DROP_OUT = 0.2
output_file = os.path.join(instruction_base_dir, "alpaca_zhtw_formatted.jsonl")

file = open(output_file, "w")

for example in alpaca_tw2tw["train"]:
    conversation = []
    # if random.random() >= SYS_DROP_OUT:
    #     conversation.append({"role": "system", "content": random.choice(SYS_PROMPT_EN2TW)})

    instruction = example["instruction"]
    if example['input'] != "":
        if not example['input'].startswith("「"):
            example['input'] = f"「{example['input']}」"
        instruction += f"\n{example['input']}"
    conversation.append({"role": "user", "content": instruction})
    conversation.append({"role":"assistant", "content": example['output']})

    example = {
        "id": hash(example["instruction"]),
        "conversations": conversation
    }
    file.write(json.dumps(example, ensure_ascii=False) + "\n")

for example in alpaca_en2tw['train']:
    conversation = []
    if random.random() >= SYS_DROP_OUT:
        conversation.append({"role": "system", "content": random.choice(SYS_PROMPT_EN2TW)})

    instruction = example["instruction"]
    if example['input'] != "":
        if not example['input'].startswith("「"):
            example['input'] = f"「{example['input']}」"
        instruction += f"\n{example['input']}"
    conversation.append({"role": "user", "content": instruction})
    conversation.append({"role": "assistant", "content": example['output']})

    example = {
        "id": hash(example["instruction"]),
        "conversations": conversation
    }
    file.write(json.dumps(example, ensure_ascii=False) + "\n")

file.close()

成語

In [9]:
import opencc
converter = opencc.OpenCC('s2tw.json')

dataset = load_dataset("KSmart/chinese_traditional_chengyu", cache_dir=".cache", split='train')
output_file = os.path.join(instruction_base_dir, "idiom.jsonl")
file = open(output_file, "w")

for example in dataset:
    example['instruction'] = converter.convert(example['instruction'])
    example['output'] = converter.convert(example['output'])
    conversation = [
        {"role": "user", "content": example['instruction']},
        {"role": "assistant", "content": example['output']}
    ]
    example = {
        "id": hash(example["instruction"]),
        "conversations": conversation
    }
    file.write(json.dumps(example, ensure_ascii=False) + "\n")

file.close()

# COCT

In [10]:
prompts = [
    "Please translate the following English paragraph into Traditional Chinese: ",
    "Translate this text into Traditional Chinese: ",
    "Convert the English below into Traditional Chinese: ",
    "Please provide a Traditional Chinese translation of the following text: ",
    "English to Traditional Chinese translation requested for the paragraph below: ",
    "Kindly render the English passage below into Traditional Chinese: ",
    "Turn the following English into Traditional Chinese: ",
    "English to Traditional Chinese: Please translate the text below: ",
    "Provide a Traditional Chinese version of the English paragraph below: ",
    "English text to be translated into Traditional Chinese follows: "
    "請將以下英文段落翻譯成繁體中文：",
    "將此文本翻譯成繁體中文：",
    "把下面的英文翻譯成繁體中文：",
    "請提供以下文本的繁體中文翻譯：",
    "請將下列英文段落翻譯為繁體中文：",
    "以下英文請翻成繁體中文：",
    "翻譯下面的英文到繁體中文：",
    "請把以下英文轉譯為繁體中文：",
    "將下列英文譯為繁體中文：",
    "英文到繁體中文：請翻譯下面的文本："
]

dataset = load_dataset("liswei/coct-en-zhtw-dedup", cache_dir=".cache")

file = open(os.path.join(instruction_base_dir, "coct_en2tw.jsonl"), "w")
for example in dataset['train']:
    if len(example['ch']) < 100 or len(example['en']) < 150:
        continue
    conversation = []

    conversation.append(
        {"role": "user", "content": f"{random.choice(prompts)}{example['en']}"})
    conversation.append({"role": "assistant", "content": example['ch']})

    example = {
        "id": hash(example["en"]),
        "conversations": conversation
    }
    file.write(json.dumps(example, ensure_ascii=False) + "\n")
file.close()

# HF Datasets

In [11]:
dataset = load_dataset("Heng666/Traditional_Chinese-aya_dataset", cache_dir='.cache/', split='train')

def convert_format(example):
    conversation = []
    conversation.append({"content": example['inputs'], "role": "user"})
    conversation.append({"content": example['targets'], "role": "assistant"})

    return {
        "id": hash(example["inputs"]),
        "conversations": conversation,
        # meta = other key/val pairs as json string
        "meta": json.dumps({k: v for k, v in example.items() if k not in ['inputs', 'targets']})
    }

dataset = dataset.map(convert_format, remove_columns=dataset.column_names, num_proc=30)
dataset.save_to_disk(os.path.join(instruction_base_dir, "aya_dataset"))

Saving the dataset (1/1 shards): 100%|██████████| 4909/4909 [00:00<00:00, 152648.49 examples/s]
